In [1]:
from sklearn.datasets import load_boston
# sklearn 버전에따라 boston housing이 제거됨, 1.1.3으로 재설치

dataset = load_boston()
print(dataset.keys())

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename', 'data_module'])


/Users/leejuhyung/opt/miniconda3/envs/AI/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Ca

In [2]:
print(dataset.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

In [3]:
import pandas as pd
dataFrame = pd.DataFrame(dataset['data'])
dataFrame.columns = dataset['feature_names']
dataFrame['target'] = dataset['target']

In [4]:
print(dataFrame.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  target  
0     15.3  396.90   4.98    24.0  
1     17.8  396.90   9.14    21.6  
2     17.8  392.83   4.03    34.7  
3     18.7  394.63   2.94    33.4  
4     18.7  396.90   5.33    36.2  


In [5]:
import torch
import torch.nn as nn

from torch.optim.adam import Adam

# 모델 정의
model = nn.Sequential(
    nn.Linear(13, 100),
    nn.ReLU(),
    nn.Linear(100, 1)
)

X = dataFrame.iloc[:, :13].values
Y = dataFrame['target'].values

batch_size = 100
lr = 0.001

optim = Adam(model.parameters(), lr=lr)

for epoch in range(200):
    for i in range(len(X)//batch_size):
        start = i*batch_size
        end = start + batch_size

        x = torch.FloatTensor(X[start:end])
        y = torch.FloatTensor(Y[start:end])

        optim.zero_grad()
        preds = model(x)
        loss = nn.MSELoss()(preds, y)
        loss.backward()
        optim.step()

    if epoch % 20 == 0:
        print(f'epoch: {epoch} loss: {loss.item()}')

/Users/leejuhyung/opt/miniconda3/envs/AI/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch: 0 loss: 890.4951782226562
epoch: 20 loss: 40.71782684326172
epoch: 40 loss: 38.555747985839844
epoch: 60 loss: 37.88384246826172
epoch: 80 loss: 37.79841232299805
epoch: 100 loss: 37.69320297241211
epoch: 120 loss: 37.61438751220703
epoch: 140 loss: 37.56703186035156
epoch: 160 loss: 37.51871109008789
epoch: 180 loss: 37.51182556152344


In [6]:
prediction = model(torch.FloatTensor(X[0, : 13]))
real = Y[0]
print(f'prediction: {prediction.item()} real: {real}')

prediction: 25.437246322631836 real: 24.0
